In [1]:
import os
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, UpSampling2D, Conv2D, Conv2DTranspose, Reshape, Input
from tensorflow.keras import optimizers
from efficientnet.tfkeras import EfficientNetB2, preprocess_input
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import cv2
import MVTecAD_loaddata as mv
import pandas as pd

res = []

def OCSVM(X, x_test, y, y_test):
    tuned_parameters = [
        {'nu': [2**-3, 2**-4, 2**-5, 2**-6, 2**-7], 'kernel': ['rbf'], 'gamma': [10**-3, 10**-4, 10**-5, 10**-6, 10**-7]},
    ]
    score = 'accuracy'
    clf = GridSearchCV(
        OneClassSVM(),
        tuned_parameters,
        cv=5,
        scoring=score)
#         scoring='%s_weighted' % score )
          
    clf.fit(X, y)
          
    a = clf.decision_function(x_test)
    b = clf.predict(x_test)
    r1 = roc_auc_score(y_test,a)
    r2 = f1_score(y_test,b)
    r3 = accuracy_score(y_test,b)
    print('roc_auc : ',r1)
    print('f1_score : ', r2)
    print('accuracy_score : ', r3)
    print('best_params : ', clf.best_params_)
    result = [r1,r2,r3]
    return result

def model_train(X_train, X_test):
    input_shape = (256, 256, 3)
    inputs = Input(input_shape)

    base_model = EfficientNetB2(include_top=False, weights='imagenet', pooling='avg')(inputs)
    x = Reshape((8, 8, 22))(base_model)
    x = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2DTranspose(64, kernel_size=(3, 3), activation='relu', padding='same', strides=2)(x)
    outputs = Conv2D(3, kernel_size=(3, 3), activation='sigmoid', padding='same')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error")

    model.fit(X_train, X_train, epochs=20, batch_size=16, shuffle=True)
    encoder = Model(model.layers[0].input, model.layers[-8].output)
    f_tr = encoder.predict(X_train, verbose=0)
    f_ts = encoder.predict(X_test, verbose=0)
    x_tr = np.reshape(f_tr, (f_tr.shape[0], -1))
    x_ts = np.reshape(f_ts, (f_ts.shape[0], -1))
    
    incov_train = np.linalg.inv(np.cov(x_tr.T))
    incov_test = np.linalg.inv(np.cov(x_ts.T))
    
    x_tr = (x_tr@incov_train)
    x_ts = (x_ts@incov_test)
    
    x_tr = x_tr/(np.max(x_tr))
    x_ts = x_ts/(np.max(x_ts))
    return x_tr, x_ts

In [2]:
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
14/14 [==============================] - 137s 8s/step - loss: 0.1319
Epoch 2/20
14/14 [==============================] - 99s 7s/step - loss: 0.1315
Epoch 3/20
14/14 [==============================] - 106s 7s/step - loss: 0.1299
Epoch 4/20
14/14 [==============================] - 99s 7s/step - loss: 0.1235
Epoch 5/20
14/14 [==============================] - 96s 7s/step - loss: 0.1052
Epoch 6/20
14/14 [==============================] - 96s 7s/step - loss: 0.0954
Epoch 7/20
14/14 [==============================] - 94s 7s/step - loss: 0.0914
Epoch 8/20
14/14 [==============================] - 93s 7s/step - loss: 0.0855
Epoch 9/20
14/14 [==============================] - 97s 7s/step - loss: 0.0739
Epoch 10/20
14/14 [==============================] - 94s 7s/step - loss: 0.0504
Epoch 11/20
14/14 [==============================] - 94s 7s/step - loss: 0.0271
Epoch 12/20
14/14 [==============================] - 94s 7s/step - loss: 0.0162
Epoch 13/20
14/14 [============================

In [3]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
14/14 [==============================] - 148s 9s/step - loss: 0.0547
Epoch 2/20
14/14 [==============================] - 128s 9s/step - loss: 0.0539
Epoch 3/20
14/14 [==============================] - 118s 8s/step - loss: 0.0512
Epoch 4/20
14/14 [==============================] - 121s 9s/step - loss: 0.0467
Epoch 5/20
14/14 [==============================] - 113s 8s/step - loss: 0.0445
Epoch 6/20
14/14 [==============================] - 112s 8s/step - loss: 0.0430
Epoch 7/20
14/14 [==============================] - 112s 8s/step - loss: 0.0410
Epoch 8/20
14/14 [==============================] - 111s 8s/step - loss: 0.0384
Epoch 9/20
14/14 [==============================] - 111s 8s/step - loss: 0.0362
Epoch 10/20
14/14 [==============================] - 113s 8s/step - loss: 0.0348
Epoch 11/20
14/14 [==============================] - 106s 8s/step - loss: 0.0335
Epoch 12/20
14/14 [==============================] - 106s 8s/step - loss: 0.0324
Epoch 13/20
14/14 [==================

In [4]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
14/14 [==============================] - 137s 8s/step - loss: 0.0932
Epoch 2/20
14/14 [==============================] - 106s 8s/step - loss: 0.0908
Epoch 3/20
14/14 [==============================] - 104s 7s/step - loss: 0.0817
Epoch 4/20
14/14 [==============================] - 104s 7s/step - loss: 0.0642
Epoch 5/20
14/14 [==============================] - 104s 7s/step - loss: 0.0395
Epoch 6/20
14/14 [==============================] - 103s 7s/step - loss: 0.0303
Epoch 7/20
14/14 [==============================] - 107s 8s/step - loss: 0.0280
Epoch 8/20
14/14 [==============================] - 109s 8s/step - loss: 0.0262
Epoch 9/20
14/14 [==============================] - 103s 7s/step - loss: 0.0252
Epoch 10/20
14/14 [==============================] - 106s 8s/step - loss: 0.0244
Epoch 11/20
14/14 [==============================] - 103s 7s/step - loss: 0.0238
Epoch 12/20
14/14 [==============================] - 105s 7s/step - loss: 0.0232
Epoch 13/20
14/14 [==================

In [5]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
18/18 [==============================] - 159s 8s/step - loss: 0.0378
Epoch 2/20
18/18 [==============================] - 134s 7s/step - loss: 0.0348
Epoch 3/20
18/18 [==============================] - 132s 7s/step - loss: 0.0258
Epoch 4/20
18/18 [==============================] - 131s 7s/step - loss: 0.0208
Epoch 5/20
18/18 [==============================] - 134s 7s/step - loss: 0.0195
Epoch 6/20
18/18 [==============================] - 133s 7s/step - loss: 0.0189
Epoch 7/20
18/18 [==============================] - 131s 7s/step - loss: 0.0186
Epoch 8/20
18/18 [==============================] - 133s 7s/step - loss: 0.0184
Epoch 9/20
18/18 [==============================] - 131s 7s/step - loss: 0.0183
Epoch 10/20
18/18 [==============================] - 131s 7s/step - loss: 0.0181
Epoch 11/20
18/18 [==============================] - 129s 7s/step - loss: 0.0181
Epoch 12/20
18/18 [==============================] - 131s 7s/step - loss: 0.0180
Epoch 13/20
18/18 [==================

In [6]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
17/17 [==============================] - 152s 8s/step - loss: 0.0291
Epoch 2/20
17/17 [==============================] - 123s 7s/step - loss: 0.0286
Epoch 3/20
17/17 [==============================] - 124s 7s/step - loss: 0.0271
Epoch 4/20
17/17 [==============================] - 124s 7s/step - loss: 0.0261
Epoch 5/20
17/17 [==============================] - 125s 7s/step - loss: 0.0258
Epoch 6/20
17/17 [==============================] - 127s 7s/step - loss: 0.0257
Epoch 7/20
17/17 [==============================] - 124s 7s/step - loss: 0.0257
Epoch 8/20
17/17 [==============================] - 124s 7s/step - loss: 0.0256
Epoch 9/20
17/17 [==============================] - 123s 7s/step - loss: 0.0256
Epoch 10/20
17/17 [==============================] - 124s 7s/step - loss: 0.0255
Epoch 11/20
17/17 [==============================] - 123s 7s/step - loss: 0.0255
Epoch 12/20
17/17 [==============================] - 123s 7s/step - loss: 0.0255
Epoch 13/20
17/17 [==================

In [7]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
25/25 [==============================] - 205s 7s/step - loss: 0.1032
Epoch 2/20
25/25 [==============================] - 181s 7s/step - loss: 0.0751
Epoch 3/20
25/25 [==============================] - 183s 7s/step - loss: 0.0300
Epoch 4/20
25/25 [==============================] - 183s 7s/step - loss: 0.0165
Epoch 5/20
25/25 [==============================] - 182s 7s/step - loss: 0.0092
Epoch 6/20
25/25 [==============================] - 181s 7s/step - loss: 0.0075
Epoch 7/20
25/25 [==============================] - 183s 7s/step - loss: 0.0070
Epoch 8/20
25/25 [==============================] - 181s 7s/step - loss: 0.0066
Epoch 9/20
25/25 [==============================] - 181s 7s/step - loss: 0.0062
Epoch 10/20
25/25 [==============================] - 181s 7s/step - loss: 0.0060
Epoch 11/20
25/25 [==============================] - 181s 7s/step - loss: 0.0058
Epoch 12/20
25/25 [==============================] - 182s 7s/step - loss: 0.0056
Epoch 13/20
25/25 [==================

In [8]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
16/16 [==============================] - 139s 7s/step - loss: 0.0501
Epoch 2/20
16/16 [==============================] - 115s 7s/step - loss: 0.0466
Epoch 3/20
16/16 [==============================] - 114s 7s/step - loss: 0.0303
Epoch 4/20
16/16 [==============================] - 117s 7s/step - loss: 0.0116
Epoch 5/20
16/16 [==============================] - 113s 7s/step - loss: 0.0070
Epoch 6/20
16/16 [==============================] - 115s 7s/step - loss: 0.0044
Epoch 7/20
16/16 [==============================] - 115s 7s/step - loss: 0.0035
Epoch 8/20
16/16 [==============================] - 115s 7s/step - loss: 0.0029
Epoch 9/20
16/16 [==============================] - 114s 7s/step - loss: 0.0025
Epoch 10/20
16/16 [==============================] - 113s 7s/step - loss: 0.0024
Epoch 11/20
16/16 [==============================] - 115s 7s/step - loss: 0.0022
Epoch 12/20
16/16 [==============================] - 116s 7s/step - loss: 0.0021
Epoch 13/20
16/16 [==================

In [9]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
14/14 [==============================] - 127s 8s/step - loss: 0.0999
Epoch 2/20
14/14 [==============================] - 105s 7s/step - loss: 0.0960
Epoch 3/20
14/14 [==============================] - 102s 7s/step - loss: 0.0789
Epoch 4/20
14/14 [==============================] - 102s 7s/step - loss: 0.0512
Epoch 5/20
14/14 [==============================] - 103s 7s/step - loss: 0.0437
Epoch 6/20
14/14 [==============================] - 103s 7s/step - loss: 0.0360
Epoch 7/20
14/14 [==============================] - 102s 7s/step - loss: 0.0276
Epoch 8/20
14/14 [==============================] - 103s 7s/step - loss: 0.0211
Epoch 9/20
14/14 [==============================] - 102s 7s/step - loss: 0.0173
Epoch 10/20
14/14 [==============================] - 102s 7s/step - loss: 0.0159
Epoch 11/20
14/14 [==============================] - 105s 7s/step - loss: 0.0151
Epoch 12/20
14/14 [==============================] - 101s 7s/step - loss: 0.0145
Epoch 13/20
14/14 [==================

In [10]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
17/17 [==============================] - 149s 8s/step - loss: 0.1248
Epoch 2/20
17/17 [==============================] - 123s 7s/step - loss: 0.1227
Epoch 3/20
17/17 [==============================] - 123s 7s/step - loss: 0.1124
Epoch 4/20
17/17 [==============================] - 125s 7s/step - loss: 0.0837
Epoch 5/20
17/17 [==============================] - 124s 7s/step - loss: 0.0443
Epoch 6/20
17/17 [==============================] - 125s 7s/step - loss: 0.0300
Epoch 7/20
17/17 [==============================] - 125s 7s/step - loss: 0.0247
Epoch 8/20
17/17 [==============================] - 124s 7s/step - loss: 0.0222
Epoch 9/20
17/17 [==============================] - 125s 7s/step - loss: 0.0204
Epoch 10/20
17/17 [==============================] - 124s 7s/step - loss: 0.0194
Epoch 11/20
17/17 [==============================] - 124s 7s/step - loss: 0.0188
Epoch 12/20
17/17 [==============================] - 124s 7s/step - loss: 0.0184
Epoch 13/20
17/17 [==================

In [11]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
20/20 [==============================] - 168s 8s/step - loss: 0.0669
Epoch 2/20
20/20 [==============================] - 150s 8s/step - loss: 0.0622
Epoch 3/20
20/20 [==============================] - 147s 7s/step - loss: 0.0382
Epoch 4/20
20/20 [==============================] - 151s 8s/step - loss: 0.0215
Epoch 5/20
20/20 [==============================] - 147s 7s/step - loss: 0.0124
Epoch 6/20
20/20 [==============================] - 149s 7s/step - loss: 0.0096
Epoch 7/20
20/20 [==============================] - 150s 7s/step - loss: 0.0086
Epoch 8/20
20/20 [==============================] - 148s 7s/step - loss: 0.0077
Epoch 9/20
20/20 [==============================] - 149s 7s/step - loss: 0.0072
Epoch 10/20
20/20 [==============================] - 148s 7s/step - loss: 0.0067
Epoch 11/20
20/20 [==============================] - 148s 7s/step - loss: 0.0061
Epoch 12/20
20/20 [==============================] - 147s 7s/step - loss: 0.0058
Epoch 13/20
20/20 [==================

In [12]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
15/15 [==============================] - 133s 7s/step - loss: 0.0179
Epoch 2/20
15/15 [==============================] - 109s 7s/step - loss: 0.0177
Epoch 3/20
15/15 [==============================] - 107s 7s/step - loss: 0.0173
Epoch 4/20
15/15 [==============================] - 107s 7s/step - loss: 0.0163
Epoch 5/20
15/15 [==============================] - 109s 7s/step - loss: 0.0157
Epoch 6/20
15/15 [==============================] - 106s 7s/step - loss: 0.0156
Epoch 7/20
15/15 [==============================] - 106s 7s/step - loss: 0.0156
Epoch 8/20
15/15 [==============================] - 106s 7s/step - loss: 0.0155
Epoch 9/20
15/15 [==============================] - 110s 7s/step - loss: 0.0155
Epoch 10/20
15/15 [==============================] - 107s 7s/step - loss: 0.0154
Epoch 11/20
15/15 [==============================] - 107s 7s/step - loss: 0.0154
Epoch 12/20
15/15 [==============================] - 107s 7s/step - loss: 0.0154
Epoch 13/20
15/15 [==================

In [13]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
4/4 [==============================] - 46s 7s/step - loss: 0.1398
Epoch 2/20
4/4 [==============================] - 30s 7s/step - loss: 0.1393
Epoch 3/20
4/4 [==============================] - 31s 7s/step - loss: 0.1389
Epoch 4/20
4/4 [==============================] - 33s 8s/step - loss: 0.1382
Epoch 5/20
4/4 [==============================] - 31s 7s/step - loss: 0.1372
Epoch 6/20
4/4 [==============================] - 31s 7s/step - loss: 0.1356
Epoch 7/20
4/4 [==============================] - 31s 7s/step - loss: 0.1329
Epoch 8/20
4/4 [==============================] - 30s 7s/step - loss: 0.1285
Epoch 9/20
4/4 [==============================] - 30s 7s/step - loss: 0.1217
Epoch 10/20
4/4 [==============================] - 30s 7s/step - loss: 0.1119
Epoch 11/20
4/4 [==============================] - 31s 8s/step - loss: 0.0993
Epoch 12/20
4/4 [==============================] - 30s 7s/step - loss: 0.0863
Epoch 13/20
4/4 [==============================] - 30s 7s/step - loss: 0.

In [14]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
14/14 [==============================] - 128s 7s/step - loss: 0.0677
Epoch 2/20
14/14 [==============================] - 102s 7s/step - loss: 0.0640
Epoch 3/20
14/14 [==============================] - 104s 7s/step - loss: 0.0484
Epoch 4/20
14/14 [==============================] - 102s 7s/step - loss: 0.0304
Epoch 5/20
14/14 [==============================] - 103s 7s/step - loss: 0.0241
Epoch 6/20
14/14 [==============================] - 108s 8s/step - loss: 0.0214
Epoch 7/20
14/14 [==============================] - 104s 7s/step - loss: 0.0199
Epoch 8/20
14/14 [==============================] - 104s 7s/step - loss: 0.0189
Epoch 9/20
14/14 [==============================] - 106s 8s/step - loss: 0.0183
Epoch 10/20
14/14 [==============================] - 106s 8s/step - loss: 0.0178
Epoch 11/20
14/14 [==============================] - 107s 8s/step - loss: 0.0173
Epoch 12/20
14/14 [==============================] - 105s 7s/step - loss: 0.0169
Epoch 13/20
14/14 [==================

In [15]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
16/16 [==============================] - 162s 8s/step - loss: 0.0216
Epoch 2/20
16/16 [==============================] - 126s 8s/step - loss: 0.0206
Epoch 3/20
16/16 [==============================] - 125s 8s/step - loss: 0.0169
Epoch 4/20
16/16 [==============================] - 128s 8s/step - loss: 0.0094
Epoch 5/20
16/16 [==============================] - 123s 8s/step - loss: 0.0066
Epoch 6/20
16/16 [==============================] - 122s 8s/step - loss: 0.0056
Epoch 7/20
16/16 [==============================] - 120s 7s/step - loss: 0.0051
Epoch 8/20
16/16 [==============================] - 116s 7s/step - loss: 0.0049
Epoch 9/20
16/16 [==============================] - 118s 7s/step - loss: 0.0047
Epoch 10/20
16/16 [==============================] - 116s 7s/step - loss: 0.0046
Epoch 11/20
16/16 [==============================] - 115s 7s/step - loss: 0.0046
Epoch 12/20
16/16 [==============================] - 114s 7s/step - loss: 0.0045
Epoch 13/20
16/16 [==================

In [16]:
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.bottle()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.cable()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.capsule()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.carpet()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.grid()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.hazelnut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.leather()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.metal_nut()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.pill()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.screw()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.tile()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.toothbrush()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.transistor()
# X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.wood()
X_train, X_test, X_test_good, X_test_error, y_tr, y_tg, y_te, y_ts = mv.zipper()
x_train, x_test = model_train(X_train, X_test)
result = OCSVM(x_train, x_test, y_tr, y_ts)
res.append(result)

Epoch 1/20
15/15 [==============================] - 143s 8s/step - loss: 0.1130
Epoch 2/20
15/15 [==============================] - 122s 8s/step - loss: 0.1096
Epoch 3/20
15/15 [==============================] - 116s 8s/step - loss: 0.0988
Epoch 4/20
15/15 [==============================] - 117s 8s/step - loss: 0.0889
Epoch 5/20
15/15 [==============================] - 117s 8s/step - loss: 0.0773
Epoch 6/20
15/15 [==============================] - 117s 8s/step - loss: 0.0689
Epoch 7/20
15/15 [==============================] - 122s 8s/step - loss: 0.0644
Epoch 8/20
15/15 [==============================] - 116s 8s/step - loss: 0.0615
Epoch 9/20
15/15 [==============================] - 117s 8s/step - loss: 0.0601
Epoch 10/20
15/15 [==============================] - 116s 8s/step - loss: 0.0594
Epoch 11/20
15/15 [==============================] - 116s 8s/step - loss: 0.0590
Epoch 12/20
15/15 [==============================] - 117s 8s/step - loss: 0.0587
Epoch 13/20
15/15 [==================

In [17]:
df = pd.DataFrame(res)
df.to_csv('DCAEOCSVM_ALL_MAH.csv')